In [16]:
import pandas as pd, numpy as np

In [17]:
# Load school directory data
school_dir_2013 = pd.read_csv('./educationdata/ccd_directory_2013.csv')
school_dir_2015 = pd.read_csv('./educationdata/ccd_directory_2015.csv')
school_dir_2017 = pd.read_csv('./educationdata/ccd_directory_2017.csv')

g12_enrollment_2013 = pd.read_csv('./educationdata/ccd_enrollment_2013.csv')
g12_enrollment_2015 = pd.read_csv('./educationdata/ccd_enrollment_2015.csv')
g12_enrollment_2017 = pd.read_csv('./educationdata/ccd_enrollment_2017.csv')

g12_retention_2013 = pd.read_csv('./educationdata/crdc_retention_2013.csv')
g12_retention_2015 = pd.read_csv('./educationdata/crdc_retention_2015.csv')
g12_retention_2017 = pd.read_csv('./educationdata/crdc_retention_2017.csv')

sat_act_participation_2013 = pd.read_csv('./educationdata/crdc_sat_act_part_2013.csv')
sat_act_participation_2015 = pd.read_csv('./educationdata/crdc_sat_act_part_2015.csv')
sat_act_participation_2017 = pd.read_csv('./educationdata/crdc_sat_act_part_2017.csv')

finance_2013 = pd.read_csv('./educationdata/crdc_school_finance_2013.csv')
finance_2015 = pd.read_csv('./educationdata/crdc_school_finance_2015.csv')
finance_2017 = pd.read_csv('./educationdata/crdc_school_finance_2017.csv')

C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=Fal

In [18]:
def clean_school_dir(df):
    # Drop the columns with nearly total missing values
    df.drop(['Unnamed: 0', 'csa', 'cbsa', 'virtual', 'state_leg_district_lower', 'state_leg_district_upper', \
             'congress_district_id', 'direct_certification', 'lunch_program', 'ncessch_num', 'bureau_indian_education', \
             'title_i_status', 'title_i_eligible', 'title_i_schoolwide', 'magnet', 'shared_time', \
             'free_lunch', 'reduced_price_lunch', 'free_or_reduced_price_lunch'], axis = 1, inplace = True)
    # Uniforming the ncessch column to 12-digit string as the key for merging the dataframes
    df.ncessch = df.ncessch.astype(str).str.zfill(12)
    df.zip_location = df.zip_location.astype(str)
    df.county_code = df.county_code.astype(str).str.zfill(5)
    # Filling missing data in state_location and zip_location by using state_mailing column
    df.state_location.replace(to_replace = '-1', value = np.nan, inplace =True) 
    df.state_location.fillna(df.state_mailing, inplace = True)
    df.zip_location.replace(to_replace = -1, value = np.nan, inplace =True) 
    df.zip_location.fillna(df.zip_mailing, inplace = True)
    # Drop states which have too much missing data...
    df.drop(df[df['state_location'] == 'AE'].index, inplace = True)
    df.drop(df[df['state_location'] == 'AP'].index, inplace = True)
    df.drop(df[df['state_location'] == 'AS'].index, inplace = True)
    df.drop(df[df['state_location'] == 'GU'].index, inplace = True)
    df.drop(df[df['state_location'] == 'PR'].index, inplace = True)
    df.drop(df[df['state_location'] == 'VI'].index, inplace = True)
    return df

school_dir_2013 = clean_school_dir(school_dir_2013)
school_dir_2015 = clean_school_dir(school_dir_2015)
school_dir_2017 = clean_school_dir(school_dir_2017)

def clean_enrollment(df):
    df = df[['ncessch', 'enrollment']]
    df.ncessch = df.ncessch.astype(str).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    return df

g12_enrollment_2013 = clean_enrollment(g12_enrollment_2013)
g12_enrollment_2015 = clean_enrollment(g12_enrollment_2015)
g12_enrollment_2017 = clean_enrollment(g12_enrollment_2017)

def clean_retention(df):
    df = df[['ncessch', 'students_retained']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    df.fillna(0, inplace = True)
    return df

g12_retention_2013 = clean_retention(g12_retention_2013)
g12_retention_2015 = clean_retention(g12_retention_2015)
g12_retention_2017 = clean_retention(g12_retention_2017)

def clean_sat_act(df):
    df = df[['ncessch', 'students_SAT_ACT']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    return df

sat_act_participation_2013 = clean_sat_act(sat_act_participation_2013)
sat_act_participation_2015 = clean_sat_act(sat_act_participation_2015)
sat_act_participation_2017 = clean_sat_act(sat_act_participation_2017)

def clean_finance(df):
    df = df[['ncessch', 'salaries_teachers', 'salaries_total', 'expenditures_nonpersonnel']]
    df.ncessch = df.ncessch.astype(str).replace('\.0', '', regex=True).str.zfill(12)
    df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
    return df

finance_2013 = clean_finance(finance_2013)
finance_2015 = clean_finance(finance_2015)
finance_2017 = clean_finance(finance_2017)


C:\Users\ymsc1\anaconda3\envs\Data Science\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\ymsc1\AppData\Local\Temp/ipykernel_16380/15968396.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['ncessch'], keep = 'first', inplace = True)
C:\Users\ymsc1\AppData\Local\Temp/ipykernel_16380/15968396.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [19]:
# merge the dataframes
def merge_school_data(df_dir, df_erl, df_rtn, df_sat, df_fnc):
    df_dir = df_dir.merge(df_erl, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_rtn, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_sat, on = 'ncessch', how = 'left')
    df_dir = df_dir.merge(df_fnc, on = 'ncessch', how = 'left')
    df_dir = df_dir.rename(columns = {'enrollment_x':'enrollment', 'enrollment_y':'g12_enrollment','students_retained':'g12_students_retained'})
# According to crdc document, missing values are encoded as -1 which should be converted to nan.
    df_dir.replace(to_replace = -1, value = np.nan, inplace =True) 
    return df_dir

school_dir_2013 = merge_school_data(school_dir_2013, g12_enrollment_2013, g12_retention_2013, sat_act_participation_2013, finance_2013)
school_dir_2015 = merge_school_data(school_dir_2015, g12_enrollment_2015, g12_retention_2015, sat_act_participation_2015, finance_2015)
school_dir_2017 = merge_school_data(school_dir_2017, g12_enrollment_2017, g12_retention_2017, sat_act_participation_2017, finance_2017)


In [20]:
# fill the missing data with earlier/later data if applicable...
school_dir_2013.set_index('ncessch', inplace = True)
school_dir_2015.set_index('ncessch', inplace = True)
school_dir_2017.set_index('ncessch', inplace = True)
school_dir_2013.update(school_dir_2015)
school_dir_2015.update(school_dir_2013)
school_dir_2015.update(school_dir_2017)
school_dir_2017.update(school_dir_2015)
school_dir_2013.reset_index(inplace = True)
school_dir_2015.reset_index(inplace = True)
school_dir_2017.reset_index(inplace = True)

In [21]:
# g12_enrollment g12_students_retained students_SAT_ACT salaries_teachers salaries_total expenditures_nonpersonnel
def fill_missing_value(df, filename):
    df.teachers_fte = df.teachers_fte.fillna(df.groupby('state_location').teachers_fte.transform('mean'))
    df.enrollment = df.enrollment.fillna(df.groupby('state_location').enrollment.transform('mean'))
    df.g12_enrollment = df.g12_enrollment.fillna(df.groupby('state_location').g12_enrollment.transform('mean'))
    df.g12_students_retained = df.g12_students_retained.fillna(df.groupby('state_location').g12_students_retained.transform('mean'))
    df.students_SAT_ACT = df.students_SAT_ACT.fillna(df.groupby('state_location').students_SAT_ACT.transform('mean'))
    df.salaries_teachers = df.salaries_teachers.fillna(df.groupby('state_location').salaries_teachers.transform('mean'))
    df.salaries_total = df.salaries_total.fillna(df.groupby('state_location').salaries_total.transform('mean'))
    df.expenditures_nonpersonnel = df.expenditures_nonpersonnel.fillna(df.groupby('state_location').expenditures_nonpersonnel.transform('mean'))
    # Save dataframes to csv files
    df.to_csv("./educationdata/" + filename + ".csv")
    return df
school_dir_2013 = fill_missing_value(school_dir_2013, 'school_dir_2013')
school_dir_2015 = fill_missing_value(school_dir_2015, 'school_dir_2015')
school_dir_2017 = fill_missing_value(school_dir_2017,'school_dir_2017')

In [22]:
school_dir_2013.isnull().sum()

ncessch                      0
year                         0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        1
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

In [23]:
school_dir_2015.isnull().sum()

ncessch                      0
year                         0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        0
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

In [24]:
school_dir_2017.isnull().sum()

ncessch                      0
year                         0
school_id                    0
school_name                  0
leaid                        0
lea_name                     0
state_leaid                  0
seasch                       0
street_mailing               0
city_mailing                 0
state_mailing                0
zip_mailing                  0
street_location              0
city_location                0
state_location               0
zip_location                 0
phone                        0
fips                         0
latitude                     0
longitude                    0
urban_centric_locale         0
county_code                  0
school_level                 0
school_type                  0
school_status                0
lowest_grade_offered         0
highest_grade_offered        0
charter                      0
teachers_fte                 0
elem_cedp                    0
high_cedp                    0
middle_cedp                  0
ungrade_

In [25]:
def create_state_data(df, filename):
# groupby state and calculate aggregated values
    df = df.groupby('state_location')
    school_stats = df.size().to_frame(name = 'school_counts')
    school_stats = (school_stats
    .join(df.agg({'enrollment':'mean'}).round(0).rename(columns = {'enrollment':'enrollment_mean'}))
    .join(df.agg({'g12_enrollment':'mean'}).round(0).rename(columns = {'g12_enrollment':'g12_enrollment_mean'}))
    .join(df.agg({'teachers_fte':'mean'}).round(0).rename(columns = {'teachers_fte':'teachers_fte_mean'}))
    .join(df.agg({'g12_students_retained':'mean'}).round(0).rename(columns = {'g12_students_retained':'g12_students_retained_mean'}))
    .join(df.agg({'students_SAT_ACT':'mean'}).round(0).rename(columns = {'students_SAT_ACT':'students_SAT_ACT_mean'}))
    .join(df.agg({'salaries_teachers':'mean'}).round(2).rename(columns = {'salaries_teachers':'salaries_teachers_mean'}))
    .join(df.agg({'salaries_total':'mean'}).round(2).rename(columns = {'salaries_total':'salaries_total_mean'}))
    .join(df.agg({'expenditures_nonpersonnel':'mean'}).round(2).rename(columns = {'expenditures_nonpersonnel':'expenditures_nonpersonnel_mean'}))
    ).reset_index()
# Save dataframes to csv files
    school_stats.to_csv("./educationdata/" + filename + ".csv")
    return school_stats

state_data_2013 = create_state_data(school_dir_2013, 'state_data_2013')
state_data_2015 = create_state_data(school_dir_2015, 'state_data_2015')
state_data_2017 = create_state_data(school_dir_2017, 'state_data_2017')
state_data_2013

,state_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,48,563.0,124.0,30.0,8.0,133.0,2158340.67,2977432.74,375157.12
1,AL,279,725.0,161.0,40.0,5.0,197.0,2084060.81,3259904.97,698079.92
2,AR,259,514.0,117.0,41.0,2.0,165.0,1789216.40,2440478.67,1066876.45
3,AZ,421,730.0,181.0,54.0,12.0,117.0,1559854.43,2537615.13,455948.45
4,CA,1253,1382.0,322.0,58.0,5.0,230.0,4278165.83,6409906.05,1069519.25
5,CO,281,793.0,187.0,42.0,14.0,182.0,2381441.62,3223684.51,3079506.14
6,CT,169,857.0,208.0,68.0,7.0,283.0,4880917.60,6721597.74,1262602.70
7,DC,25,485.0,102.0,44.0,2.0,164.0,3699626.52,6658235.10,1206458.70
8,DE,27,1073.0,240.0,69.0,7.0,242.0,4409897.89,5714188.59,1055758.81
9,FL,478,1580.0,349.0,83.0,11.0,469.0,3137591.19,4235301.23,596345.10


In [26]:
state_data_2015

,state_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,48,548.0,125.0,30.0,6.0,76.0,1948394.62,2687676.29,519521.46
1,AL,271,740.0,173.0,42.0,4.0,205.0,2103234.65,2725699.98,892533.22
2,AR,272,501.0,117.0,40.0,1.0,156.0,1853331.88,2375981.85,1581911.90
3,AZ,408,744.0,182.0,54.0,12.0,142.0,1767939.13,2689799.38,768994.37
4,CA,1283,1353.0,319.0,58.0,5.0,254.0,4500378.41,6107362.81,1163814.69
5,CO,278,828.0,196.0,44.0,11.0,202.0,2523295.68,3347631.20,687273.10
6,CT,172,841.0,203.0,70.0,6.0,249.0,5236139.65,7003150.54,1848413.61
7,DC,28,503.0,111.0,38.0,5.0,181.0,3011276.54,5037065.65,1195233.89
8,DE,29,1024.0,223.0,67.0,3.0,248.0,4982663.72,6402117.83,1986154.31
9,FL,486,1590.0,365.0,84.0,7.0,526.0,3885002.77,5517565.60,728634.21


In [27]:
state_data_2017

,state_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,56,508.0,112.0,28.0,5.0,69.0,1843139.86,2532664.41,467518.34
1,AL,292,727.0,167.0,41.0,5.0,197.0,2083518.38,2691311.50,851044.99
2,AR,276,516.0,118.0,41.0,1.0,163.0,1895425.57,2422515.09,2333468.38
3,AZ,457,727.0,174.0,52.0,12.0,135.0,1744228.42,2621916.67,842990.76
4,CA,1368,1310.0,303.0,56.0,4.0,244.0,4359428.29,5933321.78,1199936.77
5,CO,332,748.0,173.0,40.0,10.0,177.0,2289492.13,3057128.29,640929.88
6,CT,184,829.0,196.0,69.0,6.0,238.0,5181582.11,6898144.87,1893723.10
7,DC,36,522.0,104.0,38.0,5.0,172.0,3093732.83,5217973.99,1136325.91
8,DE,32,998.0,211.0,64.0,3.0,240.0,4991279.71,6378622.87,1954107.26
9,FL,599,1375.0,307.0,79.0,6.0,459.0,3497749.88,4975609.75,668524.56


In [28]:
def create_zip_data(df, filename):
# groupby state and calculate aggregated values
    df = df.groupby(['state_location', 'zip_location'])
    school_stats = df.size().to_frame(name = 'school_counts')
    school_stats = (school_stats
    .join(df.agg({'enrollment':'mean'}).round(0).rename(columns = {'enrollment':'enrollment_mean'}))
    .join(df.agg({'g12_enrollment':'mean'}).round(0).rename(columns = {'g12_enrollment':'g12_enrollment_mean'}))
    .join(df.agg({'teachers_fte':'mean'}).round(0).rename(columns = {'teachers_fte':'teachers_fte_mean'}))
    .join(df.agg({'g12_students_retained':'mean'}).round(0).rename(columns = {'g12_students_retained':'g12_students_retained_mean'}))
    .join(df.agg({'students_SAT_ACT':'mean'}).round(0).rename(columns = {'students_SAT_ACT':'students_SAT_ACT_mean'}))
    .join(df.agg({'salaries_teachers':'mean'}).round(2).rename(columns = {'salaries_teachers':'salaries_teachers_mean'}))
    .join(df.agg({'salaries_total':'mean'}).round(2).rename(columns = {'salaries_total':'salaries_total_mean'}))
    .join(df.agg({'expenditures_nonpersonnel':'mean'}).round(2).rename(columns = {'expenditures_nonpersonnel':'expenditures_nonpersonnel_mean'}))
    ).reset_index()
# Save dataframes to csv files
    school_stats.to_csv("./educationdata/" + filename + ".csv")
    return school_stats

zip_data_2013 = create_zip_data(school_dir_2013, 'zip_data_2013')
zip_data_2015 = create_zip_data(school_dir_2015, 'zip_data_2015')
zip_data_2017 = create_zip_data(school_dir_2017, 'zip_data_2017')
zip_data_2013

,state_location,zip_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,99502,1,1693.0,374.0,71.0,16.0,403.0,4749378.0,6365571.27,169781.0
1,AK,99504,1,1491.0,325.0,74.0,30.0,334.0,4488537.0,6033659.72,180609.0
2,AK,99507,1,1693.0,402.0,78.0,31.0,382.0,4873194.0,6646636.89,175642.0
3,AK,99508,1,2209.0,524.0,104.0,59.0,367.0,5836199.0,7751654.56,240688.0
4,AK,99516,1,1500.0,361.0,64.0,23.0,394.0,4401731.0,5816422.48,146653.0
...,...,...,...,...,...,...,...,...,...,...,...
12518,WY,83001,1,638.0,132.0,48.0,0.0,0.0,88924.0,88924.00,1296579.0
12519,WY,83110,1,728.0,184.0,48.0,0.0,2.0,3332525.0,4767869.65,1540815.0
12520,WY,83113,1,192.0,54.0,15.0,4.0,55.0,840230.0,1256032.31,136149.0
12521,WY,83114,1,116.0,19.0,12.0,0.0,0.0,782400.0,1115681.44,643955.0


In [29]:
zip_data_2015

,state_location,zip_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,99502,1,1652.0,419.0,77.0,12.0,293.0,4908340.0,6383338.00,178682.0
1,AK,99503,1,286.0,38.0,15.0,0.0,36.0,964545.0,1314766.75,20079.0
2,AK,99504,1,1397.0,277.0,81.0,29.0,132.0,4141399.0,5571506.50,154535.0
3,AK,99507,1,1594.0,406.0,74.0,28.0,181.0,4450604.0,6008551.50,156488.0
4,AK,99508,1,2031.0,509.0,107.0,63.0,167.0,5537667.0,7326735.00,253944.0
...,...,...,...,...,...,...,...,...,...,...,...
13201,WY,83001,2,363.0,88.0,28.0,4.0,88.0,2110720.5,3053079.50,228487.5
13202,WY,83101,2,132.0,21.0,10.0,0.0,26.0,545288.0,793218.35,57704.5
13203,WY,83110,2,385.0,94.0,26.0,0.0,130.0,1755555.5,2753885.50,86087.0
13204,WY,83113,1,185.0,50.0,14.0,1.0,29.0,940515.0,1287956.25,142224.0


In [30]:
zip_data_2017

,state_location,zip_location,school_counts,enrollment_mean,g12_enrollment_mean,teachers_fte_mean,g12_students_retained_mean,students_SAT_ACT_mean,salaries_teachers_mean,salaries_total_mean,expenditures_nonpersonnel_mean
0,AK,99502,1,1652.0,419.0,77.0,12.0,293.0,4908340.0,6383338.00,178682.0
1,AK,99503,1,286.0,38.0,15.0,0.0,36.0,964545.0,1314766.75,20079.0
2,AK,99504,2,779.0,150.0,46.0,16.0,69.0,2424280.0,3278560.12,89804.0
3,AK,99507,1,1594.0,406.0,74.0,28.0,181.0,4450604.0,6008551.50,156488.0
4,AK,99508,1,2031.0,509.0,107.0,63.0,167.0,5537667.0,7326735.00,253944.0
...,...,...,...,...,...,...,...,...,...,...,...
13860,WY,83001,2,363.0,88.0,28.0,4.0,88.0,2110720.5,3053079.50,228487.5
13861,WY,83101,2,132.0,21.0,10.0,0.0,26.0,545288.0,793218.35,57704.5
13862,WY,83110,2,385.0,94.0,26.0,0.0,130.0,1755555.5,2753885.50,86087.0
13863,WY,83113,1,185.0,50.0,14.0,1.0,29.0,940515.0,1287956.25,142224.0
